In [1]:
import numpy as np

In [2]:
def write_header_lines(file, layer_height, line_width, layer_count, mesh, start_x, start_y, start_z=0.3):
    with open(file, 'w') as f:
        header_lines = [
            f";layer_height = {layer_height}",
            f"\n;line_width = {line_width}",
            f"\n;layer_count = {layer_count}",
            f"\n;mesh = {mesh}"
        ]
        
        initialize_lines = [
            "\nG21 ;start of the code",
            "\nG1 Z15 F300",
            "\nG28 X0 Y0 ;Home",
            "\nG92 X0 Y0 ;Consider this as current",
            "\nG0 X50 Y50 F3000 ;Go-to Offset",  
            "\nG92 X0 Y0 ;Reset",
            "\n",
            f"\nG0 F3600 X{start_x:.3f} Y{start_y:.3f} Z{start_z:.3f} ;Go to start position",
            "\nM7",
            "\n\n"
        ]
        
        f.writelines(header_lines)
        f.writelines(initialize_lines)
        
        
def writ_finish_lines(file):
    with open(file, 'a') as f:
        finish_lines = [
            "\n\n;Finish",
            "\nM9",
            "\nG1 Z10.000"
            "\nG28 X-100 Y-100;Home",
        ]
        f.writelines(finish_lines)
        
def write_G1_line(delta_x, delta_y, xy, lines, pause=True):
    xy += [delta_x, delta_y]
    lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f}")
    if pause:
        lines.append(f"\nG4 P30")
    # print(xy)
    return xy, lines

def write_init_layer(xy, z, lines):
    lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f} Z{z:.3f}")
    return lines    

def draw_layer_out_in(file, xy1, xy3, z, w):
    
    xy = np.array([xy1[0] + w/2, xy1[1] + w/2])
    a = xy3[0] - xy1[0] - w
    b = xy3[1] - xy1[1] - w
    
    lines = []
    lines = write_init_layer(xy, z, lines)
    for i in range (np.min([int(a/w), int(b/w)])+1):
        if (i%2==0):
            xy, lines = write_G1_line(a-w*i, 0, xy, lines)
            xy, lines = write_G1_line(0, b-w*i, xy, lines)
        else:
            xy, lines = write_G1_line(-a+w*(i-1), 0, xy, lines)
            xy, lines = write_G1_line(0, -b+w*(i-1), xy, lines, pause=False)
            xy, lines = write_G1_line(w, w, xy, lines, pause=False)
    
    with open(file, 'a') as f:
        f.writelines(lines)
        
        
def draw_layer_in_out(file, xy1, xy3, z, w):
    
    xy = np.array([xy1[0] + w/2, xy1[1] + w/2])
    a = xy3[0] - xy1[0] - w
    b = xy3[1] - xy1[1] - w
    
    lines_inverse = []
    lines_inverse = write_init_layer(xy, z, lines_inverse)
    for i in range (np.min([int(a/w), int(b/w)])+1):
        if (i%2==0):
            xy, lines_inverse = write_G1_line(a-w*i, 0, xy, lines_inverse)
            xy, lines_inverse = write_G1_line(0, b-w*i, xy, lines_inverse)
        else:
            xy, lines_inverse = write_G1_line(-a+w*(i-1), 0, xy, lines_inverse)
            xy, lines_inverse = write_G1_line(0, -b+w*(i-1), xy, lines_inverse, pause=False)
            xy, lines_inverse = write_G1_line(w, w, xy, lines_inverse)
            
    lines = []
    lines = write_init_layer(xy, z, lines)
    l = len(lines_inverse)
    for i in range (len(lines_inverse)):
        if (lines_inverse[i].startswith('G4')):
            lines.append(lines_inverse[l-i-2])
            lines.append(lines_inverse[l-i-1])
            i += 1
        else:
            lines.append(lines_inverse[l-i-1])
            
    with open(file, 'a') as f:
        f.writelines(lines)
        
        

def draw_layer(file, xy1, xy3, z, w, direction):
    """Draw single platform layer by concentrate infill

    Args:
        xy1, xy3 (ndarray)
        w (float)          : line width
        direction (int)    : 0 for out -> in (counter clockwise) 
                             1 for in -> out (clockwise)
    """
    
    if direction == 0:
        draw_layer_out_in(file, xy1, xy3, z, w)
    elif direction == 1:
        draw_layer_in_out(file, xy1, xy3, z, w)
        
def move_to(file, x, y, z):
    lines = [
        f"\nM9"
        f"\nG1 Z10.000"
        f"\nG1 X{x:.3f} Y{y:.3f}"
        f"\nG1 Z{z:.3f}"
        f"\nM7"
    ]
    with open(file, 'a') as f:
        f.writelines(lines)

In [3]:
mesh = 'middle'
layer_height = 0.3
line_width = 0.4
layer_count = 2
length_a = 8.6
length_b = 13.5
wall_layer_count = 0
file = '7.23/middle_'+str(length_b)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

# xb_list = [np.sqrt(4*(layer_height/2+i*layer_height)) for i in range (10)]

for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-length_a/2 + xb, -length_b/2])
    xy3 = np.array([length_a/2 - xb, length_b/2])
    
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, True)
        
writ_finish_lines(file)

In [67]:
mesh = 'base'
layer_height = 0.4
line_width = 0.4
layer_count = 2

x1 = 4.3
x2 = 4.6
x3 = 15

y1 = 1
y2 = 15

file = '7.24/base_'+str(x3)+'_'+str(y2)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -x3, -y2)

# base film (2 layers):
xy1 = np.array([-x3, -y2])
xy3 = np.array([x3, y2])
draw_layer(file, xy1, xy3, 0.3, line_width, False)
draw_layer(file, xy1, xy3, 0.7, line_width, True)

# middle part low:
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-x1+xb, -y2])
    xy3 = np.array([x1-xb, -y1])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        

# middle part high:
move_to(file, -x1, y1, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-x1+xb, y1])
    xy3 = np.array([x1-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)


# left side:
move_to(file, -x3, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-x3, -y2])
    xy3 = np.array([-x2-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
# right side:
move_to(file, x2, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([x2+xb, -y2])
    xy3 = np.array([x3, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)

             
# side wall (only one for thickness):
move_to(file, -x2, -y2+line_width/2, 1.1)
lines = []
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    if i%2 == 0:
        lines.append(f"\nG1 X{-x2-xb+line_width/4} Z{i*layer_height + 1.1}")
        lines.append(f"\nG1 X{x2+xb-line_width/4}")
    else:
        lines.append(f"\nG1 X{x2+xb-line_width/4} Z{i*layer_height + 1.1}")
        lines.append(f"\nG1 X{-x2-xb+line_width/4}")
with open(file, 'a') as f:
    f.writelines(lines)
    
move_to(file, x2, y2-line_width/2, 1.1)
lines = []
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    if i%2 == 0:
        lines.append(f"\nG1 X{x2+xb-line_width/4} Z{i*layer_height + 1.1}")
        lines.append(f"\nG1 X{-x2-xb+line_width/4}")
    else:
        lines.append(f"\nG1 X{-x2-xb+line_width/4} Z{i*layer_height + 1.1}")
        lines.append(f"\nG1 X{x2+xb-line_width/4}")
with open(file, 'a') as f:
    f.writelines(lines)
    
# move_to(file, -x2, y2-line_width/2, 1.1)
# lines = []
# for i in range (int(4/layer_height)):
#     xb = np.sqrt(4*(layer_height/2+i*layer_height))
#     if i%2 == 0:
#         lines.append(f"\nG1 X{-x2-xb+line_width/4} Z{i*layer_height + 1.1}")
#         lines.append(f"\nG1 X{-x1+xb-line_width/4}")
#     else:
#         lines.append(f"\nG1 X{-x1+xb-line_width/4} Z{i*layer_height + 1.1}")
#         lines.append(f"\nG1 X{-x2-xb+line_width/4}")
# with open(file, 'a') as f:
#     f.writelines(lines)
    
# move_to(file, x2, y2-line_width/2, 1.1)
# lines = []
# for i in range (int(4/layer_height)):
#     xb = np.sqrt(4*(layer_height/2+i*layer_height))
#     if i%2 == 0:
#         lines.append(f"\nG1 X{x2+xb-line_width/4} Z{i*layer_height + 1.1}")
#         lines.append(f"\nG1 X{x1-xb+line_width/4}")
#     else:
#         lines.append(f"\nG1 X{x1-xb+line_width/4} Z{i*layer_height + 1.1}")
#         lines.append(f"\nG1 X{x2+xb-line_width/4}")
# with open(file, 'a') as f:
#     f.writelines(lines)
        
writ_finish_lines(file)

In [9]:
mesh = 'base'
layer_height = 0.4
line_width = 0.4
layer_count = 2

x1 = 4.3
x2 = 0.3
x3 = 15

y1 = 1
y2 = 10

file = '7.29/base_'+str(x3)+'_'+str(y2)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -x3, -y2)

# base film (2 layers):
xy1 = np.array([-x3, -y2])
xy3 = np.array([x3, y2])
draw_layer(file, xy1, xy3, 0.3, line_width, False)
draw_layer(file, xy1, xy3, 0.7, line_width, True)

# # middle part low:
# for i in range (int(4/layer_height)):
#     xb = np.sqrt(4*(layer_height/2+i*layer_height))
#     xy1 = np.array([-x1+xb, -y2])
#     xy3 = np.array([x1-xb, -y1])
#     if i%2 == 0:
#         draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
#     else:
#         draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        

# # middle part high:
# move_to(file, -x1, y1, 1.1)
# for i in range (int(4/layer_height)):
#     xb = np.sqrt(4*(layer_height/2+i*layer_height))
#     xy1 = np.array([-x1+xb, y1])
#     xy3 = np.array([x1-xb, y2])
#     if i%2 == 0:
#         draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
#     else:
#         draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)


# left side:
move_to(file, -x3, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-x3, -y2])
    xy3 = np.array([-x2-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
# right side:
move_to(file, x2, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([x2+xb, -y2])
    xy3 = np.array([x3, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)

             
# move_to(file, -x2, y2-line_width/2, 1.1)
# lines = []
# for i in range (int(4/layer_height)):
#     xb = np.sqrt(4*(layer_height/2+i*layer_height))
#     if i%2 == 0:
#         lines.append(f"\nG1 X{-x2-xb+line_width/4} Z{i*layer_height + 1.1}")
#         lines.append(f"\nG1 X{-x1+xb-line_width/4}")
#     else:
#         lines.append(f"\nG1 X{-x1+xb-line_width/4} Z{i*layer_height + 1.1}")
#         lines.append(f"\nG1 X{-x2-xb+line_width/4}")
# with open(file, 'a') as f:
#     f.writelines(lines)
    
# move_to(file, x2, y2-line_width/2, 1.1)
# lines = []
# for i in range (int(4/layer_height)):
#     xb = np.sqrt(4*(layer_height/2+i*layer_height))
#     if i%2 == 0:
#         lines.append(f"\nG1 X{x2+xb-line_width/4} Z{i*layer_height + 1.1}")
#         lines.append(f"\nG1 X{x1-xb+line_width/4}")
#     else:
#         lines.append(f"\nG1 X{x1-xb+line_width/4} Z{i*layer_height + 1.1}")
#         lines.append(f"\nG1 X{x2+xb-line_width/4}")
# with open(file, 'a') as f:
#     f.writelines(lines)
        
writ_finish_lines(file)

# Cube components

In [10]:
mesh = 'middle'
layer_height = 0.4
line_width = 0.4
layer_count = 2
length_a = 25
length_b = 25
wall_layer_count = 0
file = '7.25/'+mesh+'_'+str(length_a)+'_'+str(length_b)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

# xb_list = [np.sqrt(4*(layer_height/2+i*layer_height)) for i in range (10)]

for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-length_a/2 + xb, -length_b/2 + xb])
    xy3 = np.array([length_a/2 - xb, length_b/2 - xb])
    
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, True)
        
writ_finish_lines(file)

In [7]:
mesh = 'side2'
layer_height = 0.3
line_width = 0.4
layer_count = 2
length_a = 25
length_b = 25
wall_layer_count = 0
file = '7.25/'+mesh+'_'+str(length_a)+'_'+str(length_b)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

# xb_list = [np.sqrt(4*(layer_height/2+i*layer_height)) for i in range (10)]

for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-length_a/2 + xb, -length_b/2])
    xy3 = np.array([length_a/2 - xb, length_b/2])
    
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, True)
        
writ_finish_lines(file)

In [8]:
mesh = 'side1'
layer_height = 0.3
line_width = 0.4
layer_count = 2
length_a = 25
length_b = 25
wall_layer_count = 0
file = '7.25/'+mesh+'_'+str(length_a)+'_'+str(length_b)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

# xb_list = [np.sqrt(4*(layer_height/2+i*layer_height)) for i in range (10)]

for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-length_a/2, -length_b/2])
    xy3 = np.array([length_a/2 - xb, length_b/2])
    
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, True)
        
writ_finish_lines(file)

In [14]:
mesh = 'bottom_cross'
layer_height = 0.4
line_width = 0.4
layer_count = 2
wall_layer_count = 0
file = '7.25/'+mesh+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -48.5, -12.5)

lines = []
w = line_width
xy = np.array([-48.5+w/2, -12.5+w/2])
write_init_layer(xy, 0.3, lines)

while(xy[0] < -12.5):
    xy, lines = write_G1_line(0, -np.sign(xy[1])*(25-w), xy, lines)
    xy, lines = write_G1_line(w, 0, xy, lines)

xy, lines = write_G1_line(0, -np.sign(xy[1])*(25-w+36), xy, lines)
xy, lines = write_G1_line(w, 0, xy, lines)
while(xy[0] < 12.5):
    xy, lines = write_G1_line(0, -np.sign(xy[1])*(25-w+72), xy, lines)
    xy, lines = write_G1_line(w, 0, xy, lines)

xy, lines = write_G1_line(0, -np.sign(xy[1])*(25-w+36), xy, lines)
xy, lines = write_G1_line(w, 0, xy, lines)
while(xy[0] < 12.5+72):
    xy, lines = write_G1_line(0, -np.sign(xy[1])*(25-w), xy, lines)
    xy, lines = write_G1_line(w, 0, xy, lines)
    
with open(file, 'a') as f:
    f.writelines(lines)


move_to(file, -12.5, -48.5, 0.7)

lines = []
w = line_width
xy = np.array([-12.5+w/2, -48.5+w/2])
lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f}")

while(xy[1] < -12.5):
    xy, lines = write_G1_line(-np.sign(xy[0])*(25-w), 0, xy, lines)
    xy, lines = write_G1_line(0, w, xy, lines)

if xy[0] < 0:
    xy, lines = write_G1_line(25-w+72, 0, xy, lines)
else:
    xy, lines = write_G1_line(-(25-w+36), 0, xy, lines)
xy, lines = write_G1_line(0, w, xy, lines)
    
while(xy[1] < 12.5):
    xy, lines = write_G1_line(-np.sign(xy[0])*(25-w+108), 0, xy, lines)
    xy, lines = write_G1_line(0, w, xy, lines)

if xy[0] < 0:
    xy, lines = write_G1_line(25-w+36, 0, xy, lines)
else:
    xy, lines = write_G1_line(-(25-w+72), 0, xy, lines)
xy, lines = write_G1_line(0, w, xy, lines)
while(xy[1] < 12.5+36):
    xy, lines = write_G1_line(-np.sign(xy[0])*(25-w), 0, xy, lines)
    xy, lines = write_G1_line(0, w, xy, lines)
    
with open(file, 'a') as f:
    f.writelines(lines)

In [6]:
mesh = 'top_cross'
layer_height = 0.4
line_width = 0.6
layer_count = 2
wall_layer_count = 0
file = '7.26/'+mesh+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -48.5, -12.5)

lines = []
w = line_width
xy = np.array([-44.6+w/2, -8.6+w/2])
write_init_layer(xy, 0.3, lines)

while(xy[0] < -8.6):
    xy, lines = write_G1_line(0, -np.sign(xy[1])*(17.2-w), xy, lines)
    xy, lines = write_G1_line(w, 0, xy, lines)

xy, lines = write_G1_line(0, -np.sign(xy[1])*(17.2-w+36), xy, lines)
xy, lines = write_G1_line(w, 0, xy, lines)
while(xy[0] < 8.6):
    xy, lines = write_G1_line(0, -np.sign(xy[1])*(17.2-w+72), xy, lines)
    xy, lines = write_G1_line(w, 0, xy, lines)

xy, lines = write_G1_line(0, -np.sign(xy[1])*(17.2-w+36), xy, lines)
xy, lines = write_G1_line(w, 0, xy, lines)
while(xy[0] < 8.6+72):
    xy, lines = write_G1_line(0, -np.sign(xy[1])*(17.2-w), xy, lines)
    xy, lines = write_G1_line(w, 0, xy, lines)
    
with open(file, 'a') as f:
    f.writelines(lines)


move_to(file, -8.6, -44.6, 0.7)

lines = []
w = line_width
xy = np.array([-8.6+w/2, -44.6+w/2])
lines.append(f"\nG1 X{xy[0]:.3f} Y{xy[1]:.3f}")

while(xy[1] < -8.6):
    xy, lines = write_G1_line(-np.sign(xy[0])*(17.2-w), 0, xy, lines)
    xy, lines = write_G1_line(0, w, xy, lines)

if xy[0] < 0:
    xy, lines = write_G1_line(17.2-w+72, 0, xy, lines)
else:
    xy, lines = write_G1_line(-(17.2-w+36), 0, xy, lines)
xy, lines = write_G1_line(0, w, xy, lines)
    
while(xy[1] < 8.6):
    xy, lines = write_G1_line(-np.sign(xy[0])*(17.2-w+108), 0, xy, lines)
    xy, lines = write_G1_line(0, w, xy, lines)

if xy[0] < 0:
    xy, lines = write_G1_line(17.2-w+36, 0, xy, lines)
else:
    xy, lines = write_G1_line(-(17.2-w+72), 0, xy, lines)
xy, lines = write_G1_line(0, w, xy, lines)
while(xy[1] < 8.6+36):
    xy, lines = write_G1_line(-np.sign(xy[0])*(17.2-w), 0, xy, lines)
    xy, lines = write_G1_line(0, w, xy, lines)
    
with open(file, 'a') as f:
    f.writelines(lines)

In [16]:
mesh = 'middle'
layer_height = 0.4
line_width = 0.4
layer_count = 2
length_a = 25
length_b = 8.6
wall_layer_count = 0
file = '7.25/'+mesh+'_'+str(length_a)+'_'+str(length_b)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, 0.4, layer_count, mesh, -length_a/2, -length_b/2)

# xb_list = [np.sqrt(4*(layer_height/2+i*layer_height)) for i in range (10)]

for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-length_a/2 + xb/1.5, -length_b/2 + xb])
    xy3 = np.array([length_a/2 - xb/1.5, length_b/2 - xb])
    
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 0.3, line_width, True)
        
writ_finish_lines(file)

In [7]:
mesh = 'side_short'
layer_height = 0.4
line_width = 0.6
layer_count = 2
length_a = 36
length_b = 6.6
wall_layer_count = 0
file = '7.26/'+mesh+'_'+str(line_width)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -length_a/2, -length_b/2)

lines = []
w = line_width
xy = np.array([-length_a/2+w/2, -length_b/2+w/2])
write_init_layer(xy, 0.3, lines)
for i in range (int(length_b/w)):
    xb = i*w/np.sqrt(2)
    if i%2 == 0:
        xy, lines = write_G1_line(length_a-w, 0, xy, lines)
        xy, lines = write_G1_line(w/np.sqrt(2), w, xy, lines)
    else:
        xy, lines = write_G1_line(-length_a+w, 0, xy, lines)
        xy, lines = write_G1_line(w/np.sqrt(2), w, xy, lines)
        
with open(file, 'a') as f:
    f.writelines(lines)
        
writ_finish_lines(file)

In [8]:
mesh = 'side_long'
layer_height = 0.4
line_width = 0.6
layer_count = 2
length_a = 72
length_b = 6.6
wall_layer_count = 0
file = '7.26/'+mesh+'_'+str(line_width)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -length_a/2, -length_b/2)

lines = []
w = line_width
xy = np.array([-length_a/2+w/2, -length_b/2+w/2])
write_init_layer(xy, 0.3, lines)
for i in range (int(length_b/w)):
    xb = i*w/np.sqrt(2)
    if i%2 == 0:
        xy, lines = write_G1_line(length_a-w, 0, xy, lines)
        xy, lines = write_G1_line(w/np.sqrt(2), w, xy, lines)
    else:
        xy, lines = write_G1_line(-length_a+w, 0, xy, lines)
        xy, lines = write_G1_line(w/np.sqrt(2), w, xy, lines)
        
with open(file, 'a') as f:
    f.writelines(lines)
        
writ_finish_lines(file)

In [10]:
mesh = 'side_joint_short'
layer_height = 0.4
line_width = 0.6
layer_count = 2
length_a = 36
length_b = 6.6
wall_layer_count = 0
file = '7.26/'+mesh+'_'+str(line_width)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -length_a/2, -length_b/2)

theta = np.arctan(np.sqrt(2))
c = np.cos(np.pi - 2*theta)
s = np.sin(np.pi - 2*theta)

lines = []
w = line_width
xy = np.array([-length_a/2+w/2, -length_b/2+w/2])
write_init_layer(xy, 0.3, lines)
for i in range (int(length_b/w)):
    xb = i*w/np.sqrt(2)
    if i%2 == 0:
        xy, lines = write_G1_line(length_a-w, 0, xy, lines)
        xy, lines = write_G1_line((length_a-w)*c, (length_a-w)*s, xy, lines)
        xy, lines = write_G1_line(-w*(s-c/np.sqrt(2)), w*(c+s/np.sqrt(2)), xy, lines)
    else:
        xy, lines = write_G1_line(-(length_a-w)*c, -(length_a-w)*s, xy, lines)
        xy, lines = write_G1_line(-(length_a-w), 0, xy, lines)
        xy, lines = write_G1_line(-w/np.sqrt(2), w, xy, lines)
        
with open(file, 'a') as f:
    f.writelines(lines)
        
writ_finish_lines(file)

In [11]:
mesh = 'side_joint_long'
layer_height = 0.4
line_width = 0.6
layer_count = 2
length_a = 36
length_b = 6.6
wall_layer_count = 0
file = '7.26/'+mesh+'_'+str(line_width)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -length_a/2, -length_b/2)

theta = np.arctan(np.sqrt(2))
c = np.cos(np.pi - 2*theta)
s = np.sin(np.pi - 2*theta)

lines = []
w = line_width
xy = np.array([-length_a/2+w/2, -length_b/2+w/2])
write_init_layer(xy, 0.3, lines)
for i in range (int(length_b/w)):
    xb = i*w/np.sqrt(2)
    if i%2 == 0:
        xy, lines = write_G1_line(2*(length_a-w), 0, xy, lines)
        xy, lines = write_G1_line((length_a-w)*c, (length_a-w)*s, xy, lines)
        xy, lines = write_G1_line(-w*(s-c/np.sqrt(2)), w*(c+s/np.sqrt(2)), xy, lines)
    else:
        xy, lines = write_G1_line(-(length_a-w)*c, -(length_a-w)*s, xy, lines)
        xy, lines = write_G1_line(-2*(length_a-w), 0, xy, lines)
        xy, lines = write_G1_line(-w/np.sqrt(2), w, xy, lines)
        
with open(file, 'a') as f:
    f.writelines(lines)
        
writ_finish_lines(file)

In [14]:
mesh = 'base_straight'
layer_height = 0.4
line_width = 0.4
layer_count = 2

x1 = 4.3
x2 = 0.5
x3 = 15

y1 = 1
y2 = 10

file = '7.30/'+mesh+'_'+str(x3)+'_'+str(y2)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -x3, -y2)

# base film (2 layers):
xy1 = np.array([-x3, -y2])
xy3 = np.array([x3, y2])
draw_layer(file, xy1, xy3, 0.3, line_width, False)
draw_layer(file, xy1, xy3, 0.7, line_width, True)


# left side:
move_to(file, -x3, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = layer_height/2+i*layer_height
    xy1 = np.array([-x3, -y2])
    xy3 = np.array([-x2-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
# right side:
move_to(file, x2, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = layer_height/2+i*layer_height
    xy1 = np.array([x2+xb, -y2])
    xy3 = np.array([x3, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
writ_finish_lines(file)

In [21]:
mesh = 'base_circle'
layer_height = 0.4
line_width = 0.4
layer_count = 2

x1 = 4.3
x2 = 0.4
x3 = 15

y1 = 1
y2 = 10

file = '7.30/'+mesh+'_'+str(x3)+'_'+str(y2)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -x3, -y2)

# base film (2 layers):
xy1 = np.array([-x3, -y2])
xy3 = np.array([x3, y2])
draw_layer(file, xy1, xy3, 0.3, line_width, False)
draw_layer(file, xy1, xy3, 0.7, line_width, True)


# left side:
move_to(file, -x3, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(16-(4-layer_height/2-i*layer_height)**2)
    xy1 = np.array([-x3, -y2])
    xy3 = np.array([-x2-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
# right side:
move_to(file, x2, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(16-(4-layer_height/2-i*layer_height)**2)
    xy1 = np.array([x2+xb, -y2])
    xy3 = np.array([x3, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
writ_finish_lines(file)

In [22]:
mesh = 'base_quadratic'
layer_height = 0.4
line_width = 0.4
layer_count = 2

x1 = 4.3
x2 = 0.4
x3 = 15

y1 = 1
y2 = 10

file = '7.30/'+mesh+'_'+str(x3)+'_'+str(y2)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -x3, -y2)

# base film (2 layers):
xy1 = np.array([-x3, -y2])
xy3 = np.array([x3, y2])
draw_layer(file, xy1, xy3, 0.3, line_width, False)
draw_layer(file, xy1, xy3, 0.7, line_width, True)


# left side:
move_to(file, -x3, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([-x3, -y2])
    xy3 = np.array([-x2-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
# right side:
move_to(file, x2, -y2, 1.1)
for i in range (int(4/layer_height)):
    xb = np.sqrt(4*(layer_height/2+i*layer_height))
    xy1 = np.array([x2+xb, -y2])
    xy3 = np.array([x3, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
writ_finish_lines(file)

In [31]:
mesh = 'base_calabash'
layer_height = 0.4
line_width = 0.4
layer_count = 2

x1 = 4.3
x2 = 0.4
x3 = 15

y1 = 1
y2 = 10

file = '7.30/'+mesh+'_'+str(x3)+'_'+str(y2)+'_'+str(layer_height)+'.gcode'
write_header_lines(file, layer_height, line_width, layer_count, mesh, -x3, -y2)

# base film (2 layers):
xy1 = np.array([-x3, -y2])
xy3 = np.array([x3, y2])
draw_layer(file, xy1, xy3, 0.3, line_width, False)
draw_layer(file, xy1, xy3, 0.7, line_width, True)


# left side:
move_to(file, -x3, -y2, 1.1)
for i in range (int(4/layer_height)):
    if i < 3:
        xb = np.sqrt(1.44-(1.2-layer_height/2-i*layer_height)**2)
    elif i == 3:
        pass
    else:
        xb = 1.4+np.sqrt(5.76-(4-layer_height/2-i*layer_height)**2)
    xy1 = np.array([-x3, -y2])
    xy3 = np.array([-x2-xb, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
# right side:
move_to(file, x2, -y2, 1.1)
for i in range (int(4/layer_height)):
    if i < 3:
        xb = np.sqrt(1.44-(1.2-layer_height/2-i*layer_height)**2)
    elif i == 3:
        pass
    else:
        xb = 1.4+np.sqrt(5.76-(4-layer_height/2-i*layer_height)**2)
    xy1 = np.array([x2+xb, -y2])
    xy3 = np.array([x3, y2])
    if i%2 == 0:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, False)
    else:
        draw_layer(file, xy1, xy3, i*layer_height + 1.1, line_width, True)
        
        
writ_finish_lines(file)

: 

In [26]:
i

3